In [ ]:
!pip install pandahouse -q

  Preparing metadata (setup.py) ... done


In [ ]:
import pandahouse as ph

In [ ]:
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')


In [ ]:
def select(sql):
    return ph.read_clickhouse(sql, connection=connection)

В следующих задачах продолжим работать с данными мобильного приложения крупной розничной сети. В этих данных отражено то, как наше приложение устанавливают (installs), как активно в нем просматривают товары (events), как активно в нем покупают (checks), и данные о склейке устройств с логинами (devices).

Note: Далее для проверки задач используются сэмплы (кусочки) данных, поэтому результирующие таблицы на LMS могут немного отличаться от тех, что вы получите в ClickHouse. Ответы на текстовые шаги или задачи с вводом ответа основаны на таблицах в ClickHouse.

– А как в него зайти? 

Теперь описание данных:

installs — содержит данные об установках приложения по дням.

DeviceID — идентификатор устройства, на которое было установлено приложение;
InstallationDate — дата установки приложения;
InstallCost — цена установки приложения в рублях;
Platform — платформа, на которой было установлено приложение (iOS/ Android);
Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).
events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

DeviceID — идентификатор устройства, на котором используется приложение;
AppPlatform — платформа, на которой используется приложение (iOS/ Android);
EventDate — дата, за которую собрана статистика;
events — количество просмотров всех товаров за этот день у этого DeviceID.
checks — содержит данные о покупках пользователей в приложении по дням

UserID — идентификатор пользователя;
Rub — суммарный чек пользователя на дату;
BuyDate — дата, за которую собрана статистика.
Особенность приложения заключается в том, что для просмотра товаров не нужна авторизация. До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица devices:

DeviceID — идентификатор устройства;
UserID — идентификатор пользователя.

In [ ]:
sql = '''select * from installs as t limit 100'''

In [ ]:
select(sql)

,InstallationDate,InstallCost,Platform,DeviceID,Source
0,2019-03-02,0,android,7950068545577019282,Source_27
1,2019-03-17,49,android,17173992779193729517,Source_14
2,2019-04-07,56,android,9528182466778893591,Source_14
3,2019-06-25,39,android,2212531864415574595,Source_9
4,2019-04-13,0,android,6959033924999748551,Source_27
...,...,...,...,...,...
95,2019-02-24,260,android,14805103225235262649,Source_9
96,2019-08-19,0,android,9296996484823722201,Source_27
97,2019-02-25,0,android,11872793628061711268,Source_27
98,2019-04-17,111,android,12110115074809355103,Source_14


In [ ]:
sql = '''select t.Source, t.Platform,
count(distinct(t.DeviceID)) as classic_unics,
uniq(t.DeviceID) as ch_uniqs,
uniqExact(t.DeviceID) as ch_exact_uniqs

from installs as t

group by t.Source, t.Platform

order by t.Source desc, t.Platform desc'''

In [ ]:
select(sql)

In [ ]:
sql = '''select uniqExact(t.DeviceID) as ch_exact_uniqs

from installs as t

group by t.Source'''

In [ ]:
select(sql)

,ch_exact_uniqs
0,20
1,3148
2,4
3,1393
4,9
5,55458
6,1499333
7,6
8,2829
9,7051


In [ ]:
sql = '''select t.Source,
uniqExact(t.DeviceID) as ch_exact_uniqs

from installs as t

group by t.Source, t.Platform

order by t.Source desc'''

In [ ]:
select(sql)

In [ ]:
sql = '''select t.Source, t.Platform,
count(distinct(t.DeviceID)) as classic_unics,
uniq(t.DeviceID) as ch_uniqs,
uniqExact(t.DeviceID) as ch_exact_uniqs,
count(*) as rows,
avg(t.InstallCost) as avg_InstallCost,
min(t.InstallCost) as min_InstallCost,
max(t.InstallCost) as max_InstallCost,
sum(t.InstallCost) as total_source_cost,
median(t.InstallCost) as median_cost,
quantile(0.50)(t.InstallCost) as median_over_quantile
 

from installs as t

group by t.Source, t.Platform

order by t.Source desc, t.Platform desc'''

In [ ]:
select(sql)

,Source,Platform,classic_unics,ch_uniqs,ch_exact_uniqs,rows,avg_InstallCost,min_InstallCost,max_InstallCost,total_source_cost,median_cost,median_over_quantile
0,Source_9,iOS,270015,267036,270015,270015,174.420199,0,349,47096070,173.0,88.00
1,Source_9,android,2405883,2417760,2405883,2405883,174.549171,0,349,419944882,175.0,83.00
2,Source_8,iOS,1,1,1,1,170.000000,170,170,170,170.0,170.00
3,Source_8,android,1407,1407,1407,1407,178.481166,0,349,251123,183.0,91.50
4,Source_7,iOS,16094,16094,16094,16094,0.000000,0,0,0,0.0,0.00
5,Source_7,android,39130,39129,39130,39130,0.000000,0,0,0,0.0,0.00
6,Source_7,WindowsPhone,234,234,234,234,0.000000,0,0,0,0.0,0.00
7,Source_6,iOS,23078,23078,23078,23078,174.586446,0,349,4029106,176.0,86.00
8,Source_6,android,45435,45435,45435,45435,175.232992,0,349,7961711,173.0,87.00
9,Source_5,iOS,9318,9318,9318,9318,176.556879,0,349,1645157,178.0,91.00


In [ ]:
sql = '''select
t.Source,
min(t.InstallCost) as min_cost,
any(t.DeviceID) as DeviceID_any

from installs as t

group by Source
--DeviceID

limit 100'''

In [ ]:
select(sql)

,Source,min_cost,DeviceID_any
0,Source_1,23,5993400199840377898
1,Source_19,0,11784091711524054405
2,Source_10,8,2811541661517653182
3,Source_2,0,8017519808046585697
4,Source_20,0,7001472667651540739
5,Source_7,0,14802651862602892654
6,Source_14,0,8780596410086162834
7,Source_4,0,2783683170486659416
8,Source_24,0,10630120774584937042
9,Source_11,0,15057887213086106703


In [ ]:
sql = '''select t.* from checks as t
limit 20'''

In [ ]:
select(sql)

,Rub,BuyDate,UserID
0,3,2019-10-04,18446583642950580515
1,4,2019-10-04,18446535622689003675
2,0,2019-10-04,18446130411954852964
3,3,2019-10-04,18446003252714243011
4,8,2019-10-04,18445948434655311802
5,17,2019-10-04,18445927732647659917
6,2,2019-10-04,18445761122620052505
7,15,2019-10-04,18445655133428855896
8,1,2019-10-04,18445587876544434519
9,1,2019-10-04,18445586451093345117


In [ ]:
# sql = '''select 
# t.UserID,
# any(t.BuyDate) as sample_by_date,
# groupArray(t.BuyDate) as all_buy_dates

# from checks as t

# where t.Rub > 1000

# group by

# t.UserID

# limit 100'''


In [ ]:
sql = '''select t.Source, t.Platform,
count(distinct(t.DeviceID)) as classic_unics,
uniq(t.DeviceID) as ch_uniqs,
uniqExact(t.DeviceID) as ch_exact_uniqs,
count(*) as rows,
avg(t.InstallCost) as avg_InstallCost,
min(t.InstallCost) as min_InstallCost,
max(t.InstallCost) as max_InstallCost,
sum(t.InstallCost) as total_source_cost,
median(t.InstallCost) as median_cost,
quantile(0.50)(t.InstallCost) as median_over_quantile
 

from installs as t

group by t.Source, t.Platform

HAVING median_cost > 100
and rows > 10000


order by t.Source desc, t.Platform desc'''

In [ ]:
select(sql)

,Source,Platform,classic_unics,ch_uniqs,ch_exact_uniqs,rows,avg_InstallCost,min_InstallCost,max_InstallCost,total_source_cost,median_cost,median_over_quantile
0,Source_9,iOS,270015,267036,270015,270015,174.420199,0,349,47096070,173.0,173.0
1,Source_9,android,2405883,2417760,2405883,2405883,174.549171,0,349,419944882,176.0,176.0
2,Source_6,iOS,23078,23078,23078,23078,174.586446,0,349,4029106,174.0,174.0
3,Source_6,android,45435,45435,45435,45435,175.232992,0,349,7961711,173.0,173.0
4,Source_5,android,105414,105082,105414,105414,174.438433,0,349,18388253,170.0,170.0
5,Source_23,android,17015,17015,17015,17015,173.779077,0,349,2956851,173.0,173.0
6,Source_15,android,99111,98767,99111,99111,174.418026,0,349,17286745,174.0,174.0
7,Source_14,iOS,154077,154793,154077,154077,174.734178,0,349,26922518,176.0,176.0
8,Source_14,android,1345256,1360646,1345256,1345256,174.574410,0,349,234847273,175.0,175.0
